## To Do

In [91]:
# To Do
# Organize the Cleaning code into readable groups
# Document the cleaning process
# Checking datatypes of both sets
# Considering renaming any columns (LOCATION, TIME)
# Checking for data loss thorughout the cleaning process

## Requirements

In [2]:
# Importing Dependencies
import pandas as pd
import os
import csv
import chardet

In [5]:
# The WHO Mortaity Dataset

# Combine Mortality rate files into a single dataframe
dirLoc = 'Resources' #filepath of the directory
Mortality_df = pd.DataFrame()

for file in os.scandir(dirLoc):
    if "Morti" in file.path: # Only append files from the Mortality Datasets
        Mortality_df = Mortality_df.append(pd.read_csv(file.path,)) #append the data of the current mortality file being read

c:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# Reading in the OECD Meat Consumption dataset
OECD_df = pd.read_csv('Resources/OECD_Meat_Consumption.csv')

## Data Exploration

Summary:
* Scope of the datasets (time)
* Size of each dataset
* data types of each set
* fields relevant to the study
* field renaming
* Full Country Name Mapping

In [11]:
# Investigating the time the datasets encompass

# The Mortaility rate dataset
Mortality_max = Mortality_df['Year'].max()
Mortality_min = Mortality_df['Year'].min()
print(f'The latest year in the mortality dataset is {Mortality_min} and the earliest year is {Mortality_max}')

OECD_max = OECD_df['TIME'].max()
OECD_min = OECD_df['TIME'].min()
print(f'The latest time in the OECD dataset is {OECD_min} and the earliest year is {OECD_max}')

# We want to narrow the scope of our records between the year 2000 and 2020



The latest year in the mortality dataset is 1988 and the earliest year is 2020
The latest time in the OECD dataset is 1990 and the earliest year is 2029


In [94]:
# Removing records with value 9
df["Sex"].value_counts() # There are 20,701 records with the 9 value in the sex field which refers to other

# Creating new dataframe wihtout 9 values in the Sex field
df_Removed9 = df[df["Sex"] != 9]

df_Removed9["Sex"].value_counts() # 9 values removed from the original dataset

1    2358126
2    2127394
Name: Sex, dtype: int64

In [95]:
df_Removed9.head(10)

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,1400,NaN,NaN,2001,101,1000,1,7,8,332,...,95.0,NaN,NaN,NaN,NaN,0.0,8.0,NaN,NaN,NaN
1,1400,NaN,NaN,2001,101,1000,2,7,8,222,...,112.0,NaN,NaN,NaN,NaN,0.0,11.0,NaN,NaN,NaN
2,1400,NaN,NaN,2001,101,1001,1,7,8,24,...,5.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
3,1400,NaN,NaN,2001,101,1001,2,7,8,14,...,6.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
4,1400,NaN,NaN,2001,101,1002,1,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
5,1400,NaN,NaN,2001,101,1002,2,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
6,1400,NaN,NaN,2001,101,1003,1,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
7,1400,NaN,NaN,2001,101,1003,2,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
8,1400,NaN,NaN,2001,101,1004,1,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
9,1400,NaN,NaN,2001,101,1004,2,7,8,0,...,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN


In [96]:
#Removing uneccesary columns
df_excised_cols = df_Removed9[["Country","Year","Cause","Sex","Deaths1"]]

In [97]:
df_excised_cols #Looks like the process took long due to some syntax error.

,Country,Year,Cause,Sex,Deaths1
0,1400,2001,1000,1,332
1,1400,2001,1000,2,222
2,1400,2001,1001,1,24
3,1400,2001,1001,2,14
4,1400,2001,1002,1,0
...,...,...,...,...,...
610179,5020,2020,Y870,2,1
610180,5020,2020,Y871,1,3
610181,5020,2020,Y883,2,1
610182,5020,2020,Y899,1,1


In [98]:
minYear = min(df_excised_cols['Year'])
maxYear = max(df_excised_cols['Year'])
print(minYear , maxYear)
# Observing the Scope of the Mortality Dataset  (1988 - 2020)
# Scope of the Meat Consumption ddDataset is 1990 - 2029
# We can limit the size of the datsets within the years 2000 and 2020
# Need to consider ways of improving write speed of Dataframe to PSQL database
# Try batch uploading to the SQL database

1988 2020


In [99]:
# removing all records that precede 2000
df_2000 = df_excised_cols[df_excised_cols["Year"] >= 2000]
df_2000

,Country,Year,Cause,Sex,Deaths1
0,1400,2001,1000,1,332
1,1400,2001,1000,2,222
2,1400,2001,1001,1,24
3,1400,2001,1001,2,14
4,1400,2001,1002,1,0
...,...,...,...,...,...
610179,5020,2020,Y870,2,1
610180,5020,2020,Y871,1,3
610181,5020,2020,Y883,2,1
610182,5020,2020,Y899,1,1


In [100]:
df_2000.count() # There are no null values
df_2000.dtypes # Columns Cause, Sex, and Country dont seem to be the datatype we need them to be

# We can explicitly assign the string datatype to the Cause column
df_2000['Cause'] = df_2000['Cause'].astype('string')
df_2000['Country'] = df_2000['Country'].astype('string')

df_2000.dtypes # The Cause Column is now string

<ipython-input-100-b930332a3958>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2000['Cause'] = df_2000['Cause'].astype('string')
<ipython-input-100-b930332a3958>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2000['Country'] = df_2000['Country'].astype('string')


Country    string
Year        int64
Cause      string
Sex         int64
Deaths1     int64
dtype: object

## WHO Country and Cause of Death Coding
The world health organization codes the causes of death and the country of origin using their mapping which is specified in tables in the Mortality dataset documentation.
I need to map the value of the codes to our clean dataset

In [101]:
# mapping the country name to the dataset using the WHO_Country_Map.csv file
# Read up on the .map() documentation in pandas

# Reading in the WHO country map as a dictionary


Code_To_Country = {}

with open('Resources/WHO_Country_Map.csv', mode = 'r') as file:
    reader = csv.reader(file)
    Code_To_Country = {rows[0] : rows[1] for rows in reader}


In [102]:
Code_To_Country
Code_To_Country.pop('country') # removing the header from the map


#apply the Code_To_Country map to the "Country" series in df_2000
df_2000['Country'] = df_2000['Country'].map(Code_To_Country)

<ipython-input-102-947fb4adf249>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2000['Country'] = df_2000['Country'].map(Code_To_Country)


In [103]:
df_2000


,Country,Year,Cause,Sex,Deaths1
0,Seychelles,2001,1000,1,332
1,Seychelles,2001,1000,2,222
2,Seychelles,2001,1001,1,24
3,Seychelles,2001,1001,2,14
4,Seychelles,2001,1002,1,0
...,...,...,...,...,...
610179,Australia,2020,Y870,2,1
610180,Australia,2020,Y871,1,3
610181,Australia,2020,Y883,2,1
610182,Australia,2020,Y899,1,1


In [105]:
OECD_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1990,0.000,NaN
1,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1991,27.942,NaN
2,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1992,26.405,NaN
3,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1993,26.370,NaN
4,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1994,25.662,NaN


In [106]:
# Cleaning tasks for the OECD dataset

OECD_df['MEASURE'].value_counts() # Includes records in Thousand Tonnes and KG per Capita
# Suggest removing the records in Thousand tonnes

print(OECD_df['TIME'].min())
print(OECD_df['TIME'].max())
# Reduce the time range to 2000 - 2020

# irrelevant columns: Frequency and flag codes

1990
2029


In [107]:
# Cleaning the OECD dataset
OECD_df.drop(['FREQUENCY', 'Flag Codes'],axis = 1,inplace = True) # Removed the Frequency and Flag Codes Columns



In [108]:
OECD_df = OECD_df[OECD_df['TIME'] >= 2000]
OECD_df = OECD_df[OECD_df['TIME'] <= 2020]
max_year = OECD_df['TIME'].max()
min_year = OECD_df['TIME'].min()

print(f'{min_year} To {max_year}')

# Reducing the scope of time to fit the range 2000 to 2020

2000 To 2020


In [109]:
OECD_df = OECD_df[OECD_df['MEASURE'] != 'THND_TONNE']

In [110]:
OECD_df['MEASURE'].unique()

array(['KG_CAP'], dtype=object)

In [111]:
OECD_df['LOCATION'].dtypes
OECD_df['LOCATION'].astype('string')


10       AUS
11       AUS
12       AUS
13       AUS
14       AUS
        ... 
12146    GBR
12147    GBR
12148    GBR
12149    GBR
12150    GBR
Name: LOCATION, Length: 3192, dtype: string

In [112]:
# Reading in the country code map
Codes_df = pd.read_csv('Resources/ISO_Alpha3_Country_Codes.csv')
Codes_df.head()

,ountry,Alpha-3 code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND


In [113]:

ISOMap_File = 'Resources/ISO_Alpha3_Country_Codes.csv'
Alpha3_To_Country = {}

with open(ISOMap_File, 'rb') as chardetect:
    print(chardet.detect(chardetect.read()))  #Determine the encoding used in the file with chardet


with open('Resources/ISO_Alpha3_Country_Codes.csv', 'r', encoding = 'cp1252') as file:
    reader = csv.reader(file)
    Alpha3_To_Country = {row[1] : row[0] for row in reader } # read in the file using cp1252 encoding

    

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [114]:
Alpha3_To_Country.pop('Alpha-3 code') # remove the header from the dictionary

'ountry'

In [115]:
#Swap out the three letter codeswith the country name
OECD_df.head()
print(len(OECD_df['LOCATION'].unique()))

38


In [116]:
OECD_df['LOCATION'] = OECD_df['LOCATION'].map(Alpha3_To_Country)
OECD_df.rename(columns = {'LOCATION': 'Country','TIME':'Year'}, inplace=True)

In [117]:
len(OECD_df['Country'].unique())

36

In [121]:
# Joining the OECD and WHO Dataset
OECD_df #3192 records
df_2000 #4039092 records

lethalMeat_df = df_2000.merge(right=OECD_df , how='inner' , on= ['Country','Year'])

In [122]:
lethalMeat_df.head()

,Country,Year,Cause,Sex,Deaths1,INDICATOR,SUBJECT,MEASURE,Value
0,Egypt,2000,A00,1,2,MEATCONSUMP,BEEF,KG_CAP,7.700
1,Egypt,2000,A00,1,2,MEATCONSUMP,PIG,KG_CAP,0.035
2,Egypt,2000,A00,1,2,MEATCONSUMP,POULTRY,KG_CAP,7.494
3,Egypt,2000,A00,1,2,MEATCONSUMP,SHEEP,KG_CAP,1.586
4,Egypt,2000,A00,2,1,MEATCONSUMP,BEEF,KG_CAP,7.700


In [123]:
# replace NaN with values we can work with 'Missing'
len(lethalMeat_df)

5110360